In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

In [2]:
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential

Using TensorFlow backend.


In [3]:
import os
import time
import warnings
from numpy import newaxis

In [4]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
warnings.filterwarnings("ignore")

In [5]:
# Loading data by reading the file 
# Use pandas later, see usage and replicate output using pandas
def load_data(filename, seq_len, normalise_window):
    f = open(filename, 'rb').read()
    data = f.decode().split('\n')

    sequence_length = seq_len + 1
    result = []
    for index in range(len(data) - sequence_length):
        result.append(data[index: index + sequence_length])
    
    if normalise_window:
        result = normalise_windows(result)

    result = np.array(result)

    row = round(0.9 * result.shape[0])
    train = result[:int(row), :]
    np.random.shuffle(train)
    x_train = train[:, :-1]
    y_train = train[:, -1]
    x_test = result[int(row):, :-1]
    y_test = result[int(row):, -1]

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))  

    return [x_train, y_train, x_test, y_test]

In [6]:
# Normalising the data, see usage
def normalise_windows(window_data):
    normalised_data = []
    for window in window_data:
        normalised_window = [((float(p) / float(window[0]))-1) for p in window]
        normalised_data.append(normalised_window)
    return normalised_data

In [7]:
# Building model
def build_model(layers):
    model = Sequential()
    model.add(LSTM(
        input_shape = (layers[1],layers[0]),
        output_dim = layers[1],
        return_sequences=True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(
        layers[2],
        return_sequences=True))
    model.add(Dropout(0.2))
    
    model.add(Dense(
        output_dim=layers[3]))
    model.add(Activation("linear"))
    
    start = time.time()
    model.compile(loss="mse",optimizer="rmsprop")
    print(" Compilation Time : ", time.time() - start)
    return model

In [8]:
def predict_point_by_point(model,data):
    predicted = model.predict(data)
    predicted = np.reshape(predicted, (predicted.size,))
    return predicted

In [10]:
def predict_sequence_full(model, data, window_size):
    curr_frame = data[0]
    predicted = []
    for i in range(len(data)):
        predicred.append(model.predict(curr_frame[newaxis,:,:])[0][0])
        curr_frame = curr_frame[1:]
        curr_frame = np.insert(curr_frame, [window_size-1], predicted[-1], axis=0)
    return predicted

In [11]:
def predict_sequences_multiple(model, data, window_size, prediction_len):
    prediction_seqs = []
    for i in range(int(len(data)/prediction_len)):
        curr_frame = data[i*prediction_len]
        predicted = []
        for j in range(prediction_len):
            predicted.append(model.predict(curr_frame[newaxis,:,:])[0,0])
            curr_frame = curr_frame[1:]
            curr_frame = np.insert(curr_frame, [window_size-1], predicted[-1], axis=0)
        prediction_seqs.append(predicted)
    return prediction_seqs

In [12]:
def plot_results(predicted_dat, true_data):
    fig = plt.figure(facecolor='white')
    ax = fig.add_subplot(111)
    ax.plot(true_data, label = 'True Data')
    plt.plot(predicted_data, label='Prediction')
    plt.legend()
    plt.show()

In [13]:
def plot_results_multiple(predicted_data, true_data, prediction_len):
    fig = plt.figure(facecolor='white')
    ax = fig.add_subplot(111)
    ax.plot(true_data, label='True Data')
    
    for i, data in enumerate(predicted_data):
        padding  = [None for p in range(i*prediction_len)]
        plt.plot(padding + data, label = 'Prediction')
        plt.legend()
    plt.show()

In [14]:
global_start_time = time.time()

In [16]:
epochs = 5
seq_len = 50
X_train, y_train, X_test,y_test = load_data('sinwave.csv', seq_len, True)

In [20]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(4455, 50, 1)
(4455,)
(495, 50, 1)
(495,)


In [23]:
model = build_model()
# model.fit(X_train,
#           y_train,
#           batch_size = 512,
#           nb_epoch = epochs,
#           validation_split = 0.05)

TypeError: build_model() takes exactly 1 argument (0 given)

In [59]:
# Building model
# def build_model(layers):
def build_model():
    model = Sequential()
    model.add(LSTM(
        input_shape = (50,1),
#         input_shape = 50,
        output_dim = 50,
        return_sequences=True))
#     model.add(LSTM(input_shape = (50,1), output_dim = 50))
    
    
    model.add(Dropout(0.2))
    
    model.add(LSTM(
        100,
        return_sequences=True))
    model.add(Dropout(0.2))
    
    model.add(Dense(
        output_dim=1))
    model.add(Activation("linear"))
    
    start = time.time()
    model.compile(loss="mse",optimizer="rmsprop")
    print(" Compilation Time : ", time.time() - start)
    return model

In [60]:
model = build_model()

TypeError: Expected int32, got <tf.Variable 'lstm_17_W_i:0' shape=(1, 50) dtype=float32_ref> of type 'Variable' instead.